<a href="https://colab.research.google.com/github/RaIvPa/Improving-BERT-for-Biomedical-QA/blob/main/huggingface_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
import transformers
!pip install transformers[sentencepiece]

     |████████████████████████████████| 2.6 MB 4.9 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 3.3 MB 45.4 MB/s 
     |████████████████████████████████| 636 kB 40.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 5.0 MB/s 


Initial installations and importing

In [3]:
from transformers import pipeline

Importing pipeline object

In [4]:
classifier = pipeline("sentiment-analysis")
classifier("This is a demo of huggingface sentiment analysis")


[{'label': 'NEGATIVE', 'score': 0.9967288970947266}]

Demoing pipelines

    feature-extraction (get the vector representation of a text)
    fill-mask
    ner (named entity recognition)
    question-answering
    sentiment-analysis
    summarization
    text-generation
    translation
    zero-shot-classification

**Replicating the pipeline**

The pipeline API consists of three main components: pre-processing (tokenisation), model processing and post-processing.

In [5]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Tokenizer is set.

In [6]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


Raw inputs are placed into appropriate tensors.

In [7]:
from transformers import TFAutoModel

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(model_name)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['pre_classifier', 'classifier', 'dropout_19']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


DistilBERT model is downloaded.

In [8]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 16, 768)


Attributes of the final vector representation are displayed.

In [9]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)
print(outputs.logits.shape)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_38']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(2, 2)


Input processing for sequence classification

In [10]:
print(outputs.logits)

tf.Tensor(
[[-1.5606964  1.6122813]
 [ 4.169231  -3.3464475]], shape=(2, 2), dtype=float32)


Logits of the resulting outputs, to be sent through a softmax function for classification

In [11]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[4.0195383e-02 9.5980465e-01]
 [9.9945587e-01 5.4418424e-04]], shape=(2, 2), dtype=float32)


Resulting probabilities:

    First sentence: NEGATIVE: 0.0402, POSITIVE: 0.9598
    Second sentence: NEGATIVE: 0.9995, POSITIVE: 0.0005

**Models**

In [12]:
from transformers import BertConfig, TFBertModel

Importing BERT config object and TF BERT model.

In [13]:
config = BertConfig()
model = TFBertModel(config)

Instantiating an unpre-trained BERT model.

In [14]:
model = TFBertModel.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Instantiating a pre-trained BERT model from the Huggingface Model Hub.

In [15]:
sequences = [
  "What's up?",
  "How's it going?",
  "Bye!"
]

In [23]:
encoded_seq = tokenizer(sequences, padding=True, truncation=True, return_tensors="tf")
print(encoded_seq)

{'input_ids': <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
array([[ 101, 2054, 1005, 1055, 2039, 1029,  102,    0],
       [ 101, 2129, 1005, 1055, 2009, 2183, 1029,  102],
       [ 101, 9061,  999,  102,    0,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0, 0, 0, 0]], dtype=int32)>}


Sequences are tokenized and placed into a TF tensor.

In [27]:
output = model(encoded_seq)
print(output)

TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor: shape=(3, 8, 768), dtype=float32, numpy=
array([[[ 0.14343002,  0.07993479, -0.08279257, ..., -0.43631852,
          0.460606  ,  0.19914457],
        [ 0.20372595, -0.2942367 ,  0.3069538 , ...,  0.4701026 ,
          0.5114696 , -0.7703376 ],
        [ 0.48882246, -0.53040105,  1.0306301 , ..., -0.55126214,
          0.17934375, -0.18660097],
        ...,
        [ 0.10438958, -0.6251829 , -0.95057005, ...,  0.16007392,
          0.21720214, -0.13163626],
        [ 0.6923576 , -0.07949506, -0.17312509, ...,  0.22097997,
         -0.31137678, -0.17324637],
        [ 0.27137172,  0.17118849,  0.2889307 , ...,  0.02700524,
          0.1733959 , -0.591145  ]],

       [[ 0.03834113,  0.12329213, -0.10843506, ..., -0.5550318 ,
          0.31812644,  0.2467761 ],
        [-0.11480317, -0.8278152 ,  0.4284883 , ...,  0.3140459 ,
          0.6908644 , -0.17196113],
        [ 0.25782725, -0.6048599 ,  1.4070927 , ..., -0.18485047,
   

**Tokenizers**

In [28]:
from transformers import BertTokenizer

Importing BERT-based tokenizer.

In [29]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading BERT base tokenizer.

In [41]:
input_tokens = bert_tokenizer.tokenize("Hello! I am learning Huggingface!")
input_IDs = bert_tokenizer.convert_tokens_to_ids(input_tokens)
# model_inputs = tf.constant(input_IDs)

Input is changed into tokens and tokens are changed into input IDs to be placed into a tensor. model(model_inputs) won't work because the model typically expects multiple sequences.

In [37]:
decoded_tokens = bert_tokenizer.decode(input_IDs)
print(decoded_tokens)

hello! i am learning huggingface!


**Handling multiple sequences**

In [45]:
model_inputs = tf.constant([input_IDs])
output = model(model_inputs)

When the input ID list only represents one sequence, a new dimension is added on top of the input ID list, then it is converted into a tensor to be sent through the BERT model.

Batching is the act of sending multiple sequences through a model all at once. A sequence can even be batched by itself.

In [46]:
batch = [input_IDs, input_IDs]
out = model(tf.constant(batch))
print(out)

TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor: shape=(2, 8, 768), dtype=float32, numpy=
array([[[-0.4731935 , -0.06740385,  0.19640781, ..., -0.06025296,
          0.19129752,  0.68221885],
        [-0.47200462, -0.00533271,  0.24600987, ...,  0.2783224 ,
          0.00768653,  0.4849256 ],
        [-0.3818979 ,  0.20104142,  0.27111843, ...,  0.29763624,
          0.0967423 ,  0.6160018 ],
        ...,
        [-0.21830142,  0.26519313,  0.4725915 , ...,  0.24766651,
          0.16676232,  0.6867252 ],
        [-0.4048373 ,  0.09211309,  0.07792024, ...,  0.21004158,
         -0.05129546,  0.25325975],
        [-0.4104473 , -0.06776839,  0.4429866 , ...,  0.37178555,
          0.07071532,  0.3052259 ]],

       [[-0.4731935 , -0.06740385,  0.19640781, ..., -0.06025296,
          0.19129752,  0.68221885],
        [-0.47200462, -0.00533271,  0.24600987, ...,  0.2783224 ,
          0.00768653,  0.4849256 ],
        [-0.3818979 ,  0.20104142,  0.27111843, ...,  0.29763624,
   

In [49]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id]
]

print(model(tf.constant(sequence1_ids)).logits)
print(model(tf.constant(sequence2_ids)).logits)
print(model(tf.constant(batched_ids)).logits)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_132']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tf.Tensor([[ 1.5693673 -1.3894573]], shape=(1, 2), dtype=float32)
tf.Tensor([[ 0.5803025 -0.4125263]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[ 1.5693673 -1.3894577]
 [ 1.3373486 -1.2163193]], shape=(2, 2), dtype=float32)


Sequences that do not have the same length need padding to compensate for the difference. This allows them to be placed into a tensor. The tokenizer's padding token ID is found through *tokenizer.pad_token_id*.

In [50]:
attention_mask = [
  [1, 1, 1],
  [1, 1, 0]
]

outputs = model(tf.constant(batched_ids), attention_mask=tf.constant(attention_mask))
print(outputs.logits)

tf.Tensor(
[[ 1.5693673  -1.3894577 ]
 [ 0.5803034  -0.41252705]], shape=(2, 2), dtype=float32)


If not addressed, models will contextualise even the padding tokens. The model needs to be instructed to ignore the padding tokens by their ID.

In [ ]:
sequence = "Hello! I like Huggingface!"
ids = tokenizer(sequence)
# padding, truncation and returned tensor type
ids = tokenizer(sequence, padding="longest", truncation=True, return_tensors="tf")

Using tokenizer() handles all of the above processes, such as input IDs and placing into tensors, and even handles padding, tensor type returned and truncation (for sequences that exceed max length).